## Capstone project

## Read in the new sample file from Radhika with only 10 HCPCS codes

In [229]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
plt.style.use('ggplot')
%matplotlib inline

In [230]:
hcpcs10 = pd.read_csv('top10_hcpcs.csv', index_col=0)

# hcpcs10 = pd.read_csv('top10_hcpcs.csv', index_col=0)
# hcpcs10 = hcpcs10.reset_index()

C:\Users\aparn\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\aparn\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [231]:
pd.set_option("display.max_columns",500)
hcpcs10.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98


In [232]:
hcpcs10.dtypes

npi                                   int64
nppes_provider_last_org_name         object
nppes_provider_first_name            object
nppes_provider_mi                    object
nppes_credentials                    object
nppes_provider_gender                object
nppes_entity_code                    object
nppes_provider_street1               object
nppes_provider_street2               object
nppes_provider_city                  object
nppes_provider_zip                   object
nppes_provider_state                 object
nppes_provider_country               object
provider_type                        object
medicare_participation_indicator     object
place_of_service                     object
hcpcs_code                           object
hcpcs_description                    object
hcpcs_drug_indicator                 object
line_srvc_cnt                       float64
bene_unique_cnt                       int64
bene_day_srvc_cnt                     int64
average_Medicare_allowed_amt    

In [233]:
hcpcs10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520985 entries, 6 to 9847441
Data columns (total 28 columns):
npi                                 1520985 non-null int64
nppes_provider_last_org_name        1520959 non-null object
nppes_provider_first_name           1511915 non-null object
nppes_provider_mi                   1075160 non-null object
nppes_credentials                   1455318 non-null object
nppes_provider_gender               1511939 non-null object
nppes_entity_code                   1520985 non-null object
nppes_provider_street1              1520983 non-null object
nppes_provider_street2              632192 non-null object
nppes_provider_city                 1520985 non-null object
nppes_provider_zip                  1520985 non-null object
nppes_provider_state                1520985 non-null object
nppes_provider_country              1520985 non-null object
provider_type                       1520985 non-null object
medicare_participation_indicator    1520985 non-n

In [234]:
hcpcs10.columns[hcpcs10.isnull().any()].tolist()

['nppes_provider_last_org_name',
 'nppes_provider_first_name',
 'nppes_provider_mi',
 'nppes_credentials',
 'nppes_provider_gender',
 'nppes_provider_street1',
 'nppes_provider_street2']

In [235]:
## Get a count of the missings by column.

len(hcpcs10.index) - hcpcs10.count()

npi                                      0
nppes_provider_last_org_name            26
nppes_provider_first_name             9070
nppes_provider_mi                   445825
nppes_credentials                    65667
nppes_provider_gender                 9046
nppes_entity_code                        0
nppes_provider_street1                   2
nppes_provider_street2              888793
nppes_provider_city                      0
nppes_provider_zip                       0
nppes_provider_state                     0
nppes_provider_country                   0
provider_type                            0
medicare_participation_indicator         0
place_of_service                         0
hcpcs_code                               0
hcpcs_description                        0
hcpcs_drug_indicator                     0
line_srvc_cnt                            0
bene_unique_cnt                          0
bene_day_srvc_cnt                        0
average_Medicare_allowed_amt             0
average_sub

In [189]:
hcpcs10.loc[:,['hcpcs_code','hcpcs_description','line_srvc_cnt','average_submitted_chrg_amt','average_Medicare_payment_amt']]

**Clean up the Zip Code variable**

In [236]:
## Check the length of the zip code variable in the HCPCS10 file

hcpcs10['zip_len']= hcpcs10["nppes_provider_zip"].str.len()

In [237]:
## Lots of variation in lengths.

hcpcs10['zip_len'].value_counts()

9.0     462774
5.0      62632
8.0      37006
4.0       6623
7.0        898
3.0        749
10.0        16
6.0          8
1.0          4
12.0         2
2.0          1
Name: zip_len, dtype: int64

In [238]:
# to get the first 5 digits of zip code, first convert the zip code var to string. 

hcpcs10["nppes_provider_zip"]= hcpcs10["nppes_provider_zip"].astype(str) 

In [239]:
# Now use string slicing to get the first five digits.

hcpcs10['prov_zip'] = hcpcs10.nppes_provider_zip.str.slice(0,5,1)

In [240]:
hcpcs10['nzip_len']= hcpcs10["prov_zip"].str.len()

In [241]:
hcpcs10['nzip_len'].value_counts()

5    1499547
4      19361
3       2064
1         11
2          2
Name: nzip_len, dtype: int64

In [242]:
## Since a bunch of zips below 5 in length remain, put in leading zeros. Ok for 4 digit. May want to drop the 3,2,1s

hcpcs10['zip_new']= hcpcs10["prov_zip"].apply(lambda x: x.zfill(5))

In [243]:
hcpcs10['nzip2_len']= hcpcs10["zip_new"].str.len()

In [244]:
hcpcs10['nzip2_len'].value_counts()

5    1520985
Name: nzip2_len, dtype: int64

In [245]:
hcpcs10.head()

**Divide the file into 2 DFs -- one for individuals and the other for organizations**

In [246]:
hcpcs10['nppes_entity_code'].nunique()

2

In [247]:
hcpcs10['nppes_entity_code'].value_counts()

I    1511939
O       9046
Name: nppes_entity_code, dtype: int64

**Get the individual level file and check it**

In [248]:
hcpcs10_ind = hcpcs10[hcpcs10['nppes_entity_code']=='I']

In [249]:
type(hcpcs10_ind)

pandas.core.frame.DataFrame

In [250]:
hcpcs10_ind.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,zip_len,prov_zip,nzip_len,zip_new,nzip2_len
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,NaN,21502,5,21502,5
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,NaN,21502,5,21502,5
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,NaN,21502,5,21502,5
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,NaN,21502,5,21502,5
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,NaN,43623,5,43623,5


In [251]:
# Obtain a list of cols with missings

hcpcs10_ind.columns[hcpcs10_ind.isnull().any()].tolist()

['nppes_provider_last_org_name',
 'nppes_provider_first_name',
 'nppes_provider_mi',
 'nppes_credentials',
 'nppes_provider_street1',
 'nppes_provider_street2',
 'zip_len']

In [252]:
# Get the count of missings by columns in df

len(hcpcs10_ind.index) - hcpcs10_ind.count()

npi                                      0
nppes_provider_last_org_name            26
nppes_provider_first_name               24
nppes_provider_mi                   436779
nppes_credentials                    56621
nppes_provider_gender                    0
nppes_entity_code                        0
nppes_provider_street1                   2
nppes_provider_street2              880777
nppes_provider_city                      0
nppes_provider_zip                       0
nppes_provider_state                     0
nppes_provider_country                   0
provider_type                            0
medicare_participation_indicator         0
place_of_service                         0
hcpcs_code                               0
hcpcs_description                        0
hcpcs_drug_indicator                     0
line_srvc_cnt                            0
bene_unique_cnt                          0
bene_day_srvc_cnt                        0
average_Medicare_allowed_amt             0
average_sub

In [68]:
## View the provider last name and first name
## Using head as shown in next box is more efficient.

hcpcs10_ind.loc[:,['npi','nppes_provider_last_org_name','nppes_provider_first_name']]

In [253]:
hcpcs10_ind[['npi','nppes_provider_last_org_name','nppes_provider_first_name']].head(10)

,npi,nppes_provider_last_org_name,nppes_provider_first_name
6,1003000126,ENKESHAFI,ARDALAN
7,1003000126,ENKESHAFI,ARDALAN
8,1003000126,ENKESHAFI,ARDALAN
10,1003000126,ENKESHAFI,ARDALAN
39,1003000142,KHALIL,RASHID
40,1003000142,KHALIL,RASHID
41,1003000142,KHALIL,RASHID
48,1003000407,GIRARDI,DAVID
51,1003000407,GIRARDI,DAVID
52,1003000407,GIRARDI,DAVID


In [254]:
hcpcs10_ind['nppes_provider_last_org_name'].value_counts()

PATEL               9159
SMITH               7574
LEE                 5437
JOHNSON             5431
MILLER              4762
SHAH                4186
BROWN               4180
WILLIAMS            3908
JONES               3847
KHAN                3581
KIM                 3414
NGUYEN              3315
SINGH               3010
DAVIS               3008
ANDERSON            3005
THOMAS              2863
WILSON              2611
MARTIN              2476
MOORE               2319
TAYLOR              2167
GUPTA               2091
THOMPSON            2090
CHEN                2033
HARRIS              1935
WHITE               1928
YOUNG               1883
SHARMA              1863
AHMED               1854
REDDY               1850
LEWIS               1813
                    ... 
FARD                   1
TRUTNAU                1
STRICH                 1
VAHI                   1
MENZOIAN               1
AJEENA                 1
SUAREZ-MEDEROS         1
EVELSON                1
BOLOURI                1


**Create the organization level dataframe and check it**

In [255]:
## Subset the organization entities to examine them closely, independent of individuals.

hcpcs10_org = hcpcs10[hcpcs10['nppes_entity_code']=='O']

In [256]:
hcpcs10_org.head()

In [257]:
hcpcs10_org.shape

(9046, 33)

In [258]:
# list of columns that have missing values

hcpcs10_org.columns[hcpcs10_org.isnull().any()].tolist()

['nppes_provider_first_name',
 'nppes_provider_mi',
 'nppes_credentials',
 'nppes_provider_gender',
 'nppes_provider_street2',
 'zip_len']

In [259]:
# number of missing values by column

len(hcpcs10_org.index) - hcpcs10_org.count()

npi                                    0
nppes_provider_last_org_name           0
nppes_provider_first_name           9046
nppes_provider_mi                   9046
nppes_credentials                   9046
nppes_provider_gender               9046
nppes_entity_code                      0
nppes_provider_street1                 0
nppes_provider_street2              8016
nppes_provider_city                    0
nppes_provider_zip                     0
nppes_provider_state                   0
nppes_provider_country                 0
provider_type                          0
medicare_participation_indicator       0
place_of_service                       0
hcpcs_code                             0
hcpcs_description                      0
hcpcs_drug_indicator                   0
line_srvc_cnt                          0
bene_unique_cnt                        0
bene_day_srvc_cnt                      0
average_Medicare_allowed_amt           0
average_submitted_chrg_amt             0
average_Medicare

In [260]:
hcpcs10_org['nppes_provider_last_org_name'].unique().tolist()

['MEDICWEST AMBULANCE, INC.',
 'MURRAYVILLE WOODSON EMERGENCY AMBULANCE SERVICE, INC.',
 "NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",
 'EAST HILLS AMBULANCE, INC',
 'CITY OF WESTERVILLE',
 'RICHMOND COUNTY BOARD OF SUPERVISORS',
 'ADVANCED SURGICAL CARE OF BATON ROUGE,LLC',
 'CHESTER TOWNSHIP TRUSTEES',
 'STONE COUNTY HOSPITAL, INC',
 'OHIO AMBULANCE SOLUTIONS, LLC',
 'WASC LLC',
 'DHEW IND HLTH SV HLTH SVS & MNTL HLTH ADM',
 'CITY OF SHEBOYGAN',
 'BEAUMONT MEDICAL TRANSPORTATION SERVICES, INC',
 'EMS DIRECT LLC',
 'FREDERICKTOWN COMMUNITY JOINT EMERGENCY AMBULANCE DISTRICT',
 'VILLAGE OF LORDSTOWN',
 'MOUNTAIN WEST SURGERY CENTER, LLC',
 'DELTA RESPONSE TEAM',
 'KING WILLIAM COUNTY',
 'CITY OF WARREN',
 'HEALTHFLEET AMBULANCE, INC.',
 'CANDESCENT EYE HEALTH SURGICENTER, LLC',
 'CITY OF BELLA VISTA AMBULANCE SERVICE',
 'HOCKING TOWNSHIP',
 'HARTFORD VOLUNTEER FIRE DEPARTMENT INC',
 'GREAT LAKES SURGERY CENTER LLC',
 'EASTON EMERGENCY SQUAD',
 'SURGERY CENTER OF MIDWEST CITY, LLC',
 '

In [261]:
hcpcs10_org['nppes_provider_last_org_name'].value_counts()

AMERICAN MEDICAL RESPONSE AMBULANCE SERVICE INC             17
PACIFIC CATARACT AND LASER INSTITUTE INC PC                 16
SOUTHWESTERN EYE CENTER LTD                                 13
LIFEGUARD AMBULANCE SERVICE LLC                             13
AMERICAN MEDICAL RESPONSE WEST                              12
HEARTLAND EMS INC                                            8
BARNET DULANEY PERKINS EYE CENTER, PLLC                      8
MED-TRANS CORPORATION                                        8
AMERICAN MEDICAL RESPONSE OF COLORADO INC                    5
AMERICAN MEDICAL RESPONSE NORTHWEST INC                      5
NATIONAL PARK SERVICE                                        5
RURAL METRO CORPORATION                                      4
PCI GROUP LLC                                                4
CITY OF FRANKLIN                                             4
TOWN OF WINDHAM                                              4
WASHINGTON TOWNSHIP                                    

**Make changes to the org name string**

In [262]:
## Convert Org last name to lower case

hcpcs10_org['nppes_provider_last_org_name_1'] = hcpcs10_org['nppes_provider_last_org_name'].apply(lambda x: x.lower())

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [263]:
## Remove punctuations etc from org name
## import string -- for later

hcpcs10_org['nppes_provider_last_org_name_1'] = hcpcs10_org['nppes_provider_last_org_name_1'].str.replace(r'[^\w\s]+', '')

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [264]:
## output shows the new org name col has been converted to lower case and the punctuation has been removed.

hcpcs10_org['nppes_provider_last_org_name_1'].unique().tolist()

['medicwest ambulance inc',
 'murrayville woodson emergency ambulance service inc',
 'northbay physicians surgery center llc',
 'east hills ambulance inc',
 'city of westerville',
 'richmond county board of supervisors',
 'advanced surgical care of baton rougellc',
 'chester township trustees',
 'stone county hospital inc',
 'ohio ambulance solutions llc',
 'wasc llc',
 'dhew ind hlth sv hlth svs  mntl hlth adm',
 'city of sheboygan',
 'beaumont medical transportation services inc',
 'ems direct llc',
 'fredericktown community joint emergency ambulance district',
 'village of lordstown',
 'mountain west surgery center llc',
 'delta response team',
 'king william county',
 'city of warren',
 'healthfleet ambulance inc',
 'candescent eye health surgicenter llc',
 'city of bella vista ambulance service',
 'hocking township',
 'hartford volunteer fire department inc',
 'great lakes surgery center llc',
 'easton emergency squad',
 'surgery center of midwest city llc',
 'minnesota eye instit

**Test if stop words can be removed from the name string**

In [265]:
## Note that the stop words list is in lower case, so this step can't be done until the prev. two steps have been done.

from nltk.corpus import stopwords
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [266]:
hcpcs10_org['nppes_provider_last_org_name_1'] = hcpcs10_org['nppes_provider_last_org_name_1'].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) 

##This splits each string to a list element. This is also called tokenization. It will return a tuple. 

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [267]:
type(hcpcs10_org['nppes_provider_last_org_name_1'])

pandas.core.series.Series

In [268]:
## This output shows that the stop words have been removed

hcpcs10_org['nppes_provider_last_org_name_1'].unique().tolist()

['medicwest ambulance inc',
 'murrayville woodson emergency ambulance service inc',
 'northbay physicians surgery center llc',
 'east hills ambulance inc',
 'city westerville',
 'richmond county board supervisors',
 'advanced surgical care baton rougellc',
 'chester township trustees',
 'stone county hospital inc',
 'ohio ambulance solutions llc',
 'wasc llc',
 'dhew ind hlth sv hlth svs mntl hlth adm',
 'city sheboygan',
 'beaumont medical transportation services inc',
 'ems direct llc',
 'fredericktown community joint emergency ambulance district',
 'village lordstown',
 'mountain west surgery center llc',
 'delta response team',
 'king william county',
 'city warren',
 'healthfleet ambulance inc',
 'candescent eye health surgicenter llc',
 'city bella vista ambulance service',
 'hocking township',
 'hartford volunteer fire department inc',
 'great lakes surgery center llc',
 'easton emergency squad',
 'surgery center midwest city llc',
 'minnesota eye institute surgery center llc',


In [269]:
hcpcs10_org.head()

In [270]:
temporg = hcpcs10_org['nppes_provider_last_org_name_1'].str.split(' ', expand=True).rename(columns = lambda x: "ORG"+str(x+1))

In [271]:
type(temporg)
temporg.head()
temporg.shape

(9046, 10)

In [272]:
## Join the temporg dataframe with the hcpcs10 df

hcpcs10_org = hcpcs10_org.join(temporg)

In [273]:
hcpcs10_org.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,zip_len,prov_zip,nzip_len,zip_new,nzip2_len,nppes_provider_last_org_name_1,ORG1,ORG2,ORG3,ORG4,ORG5,ORG6,ORG7,ORG8,ORG9,ORG10
844,1003006180,"MEDICWEST AMBULANCE, INC.",NaN,NaN,NaN,NaN,O,9 W DELHI AVE,NaN,NORTH LAS VEGAS,890327836,NV,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,11857.0,7355,11824,441.111760,1020.810443,330.997026,322.372015,1.210375e+07,3.924632e+06,NaN,89032,5,89032,5,medicwest ambulance inc,medicwest,ambulance,inc,None,None,None,None,None,None,None
2692,1003015355,MURRAYVILLE WOODSON EMERGENCY AMBULANCE SERVIC...,NaN,NaN,NaN,NaN,O,1930 STATE HIGHWAY 267,NaN,MURRAYVILLE,62668,IL,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,18.0,15,18,419.096667,733.333333,328.568889,335.160000,1.320000e+04,5.914240e+03,NaN,62668,5,62668,5,murrayville woodson emergency ambulance servic...,murrayville,woodson,emergency,ambulance,service,inc,None,None,None,None
2938,1003015843,"NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",NaN,NaN,NaN,NaN,O,1006 NUT TREE ROAD,NaN,VACAVILLE,95687,CA,US,Ambulatory Surgical Center,Y,F,66984,Removal of cataract with insertion of lens,N,318.0,206,318,1317.601509,9253.144654,1023.232421,747.722201,2.942500e+06,3.253879e+05,NaN,95687,5,95687,5,northbay physicians surgery center llc,northbay,physicians,surgery,center,llc,None,None,None,None,None
6245,1003035213,"EAST HILLS AMBULANCE, INC",NaN,NaN,NaN,NaN,O,3111 ELTON RD,NaN,JOHNSTOWN,159042731,PA,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,469.0,327,469,410.967463,947.547974,313.595373,326.508316,4.444000e+05,1.470762e+05,NaN,15904,5,15904,5,east hills ambulance inc,east,hills,ambulance,inc,None,None,None,None,None,None
8493,1003047390,CITY OF WESTERVILLE,NaN,NaN,NaN,NaN,O,400 W MAIN ST,NaN,WESTERVILLE,430811436,OH,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,672.0,547,672,406.070000,815.120000,317.138393,334.723378,5.477606e+05,2.131170e+05,NaN,43081,5,43081,5,city westerville,city,westerville,None,None,None,None,None,None,None,None


**Work with HCPCS10 address fields**

In [274]:
hcpcs10_org['nppes_provider_street1'].unique().tolist()

['9 W DELHI AVE',
 '1930 STATE HIGHWAY 267',
 '1006 NUT TREE ROAD',
 '3111 ELTON RD',
 '400 W MAIN ST',
 '101 COURT CIRCLE',
 '7310 PERKINS ROAD',
 '5580 ST RT 380',
 '1434 CENTRAL AVE E',
 '26420 LAKELAND BLVD',
 '3373 COMMERCE PARKWAY',
 '4212 NORTH 16TH STREET',
 '1326 N 25TH ST',
 '25400 W 8 MILE RD',
 '316-B SOUTH 6TH STREET',
 '139 COLUMBUS RD',
 '1595 SALT SPRINGS RD',
 '1551 SOUTH RENAISSANCE TOWNE DR',
 '698 JONES ST',
 '180 HORSE LANDING ROAD #4',
 '1 CITY SQ',
 '235 NEW YORK DR',
 '51 STATE ROAD',
 '103 TOWN CTR W',
 '1175 CINCINNATI ZANESVILLE RD SW',
 '180 SMITH STREET',
 '2848 NILES RD',
 '908 PACKER ST',
 '8121 NATIONAL AVE',
 '3401 S BROADWAY',
 '29 ELM ST',
 '225 PARK ST',
 '200 S MIAMI AVE',
 '402 MERCER ST',
 '67 E CLARK ST',
 '319 STANFORD ST',
 '110 N MAIN ST',
 '606 W POTTER AVE',
 '1450 ROSS CLARK CIR',
 '100 S WILLIAMS ST',
 '2200 HUNTINGTON DR',
 '895 MAIN ST',
 '100 E CHARLOTTE ST',
 '13210 GOODLAND ST',
 '6591 ORANGE DR',
 '1000 BROADWAY DRIVE',
 '333 W. LAKE

In [275]:
hcpcs10_org['nppes_provider_street2'].unique().tolist()

[nan,
 'SUITE 1',
 'PHOENIX INDIAN MEDICAL CENTER AMBULATORY SURGICAL CENT',
 'SUITE 20',
 'SUITE 200',
 'SUITE B',
 'SUITE 108',
 'SUITE 4',
 'SUITE 46',
 'BOX 538',
 '902 US HWY 41 NW',
 'SUITE 800',
 'BOX 205',
 'SUITE 100',
 'STE. U',
 'SUITE 120',
 'SUITE 104',
 'HC 63 - BOX 133',
 '1 KENNEDY PLAZA',
 'SUITE 110',
 'STE 950',
 '65 AVENUE',
 'SUITE 11B',
 'MAIN FIRE STATION',
 'SUITE 400',
 'SUITE 10E',
 '14 WILSON DRIVE',
 'STE 400',
 'STATION ONE',
 'SUITE 330',
 'SUITE 124',
 'SUITE D',
 'SUITE 150',
 'STE 102',
 'ROOM 108',
 'SUITE 130',
 'SUITE 101',
 'SUITE 1515',
 'STE 100',
 'STE 204',
 'SUITE 2',
 'BLDG 7',
 'SUITE 5',
 'SUITE 160',
 'PULASKI COUNTY AMBULANCE',
 'SUITE 107',
 'SPACE #225',
 'HASKELL COUNTY COURT HOUSE',
 'SUITE 500',
 'SUITE 103',
 'SUITE 180',
 'SUITE 2000',
 'BYPASS N',
 'SUITE 118',
 'STE 2',
 'SUITE 315',
 'SUITE 102',
 'SUITE C 2020',
 'SUITE A',
 'SUITE S4',
 'STE 202A',
 '#11',
 'STE A',
 'STE 303',
 '10TH FLOOR, SUITE 1000',
 'SUITE C',
 'SUITE 105

**Make changes to the street1 variable to enable easier match with LEIE**

In [276]:
## Convert the address field to lower case

hcpcs10_org['nppes_provider_street1']= hcpcs10_org['nppes_provider_street1'].apply(lambda x: x.lower())

In [277]:
## Remove punctuations from address field

hcpcs10_org['nppes_provider_street1'] = hcpcs10_org['nppes_provider_street1'].str.replace(r'[^\w\s]+', '')

In [278]:
hcpcs10_org.head()

hcpcs10_org['nppes_provider_street1'].unique().tolist()

['9 w delhi ave',
 '1930 state highway 267',
 '1006 nut tree road',
 '3111 elton rd',
 '400 w main st',
 '101 court circle',
 '7310 perkins road',
 '5580 st rt 380',
 '1434 central ave e',
 '26420 lakeland blvd',
 '3373 commerce parkway',
 '4212 north 16th street',
 '1326 n 25th st',
 '25400 w 8 mile rd',
 '316b south 6th street',
 '139 columbus rd',
 '1595 salt springs rd',
 '1551 south renaissance towne dr',
 '698 jones st',
 '180 horse landing road 4',
 '1 city sq',
 '235 new york dr',
 '51 state road',
 '103 town ctr w',
 '1175 cincinnati zanesville rd sw',
 '180 smith street',
 '2848 niles rd',
 '908 packer st',
 '8121 national ave',
 '3401 s broadway',
 '29 elm st',
 '225 park st',
 '200 s miami ave',
 '402 mercer st',
 '67 e clark st',
 '319 stanford st',
 '110 n main st',
 '606 w potter ave',
 '1450 ross clark cir',
 '100 s williams st',
 '2200 huntington dr',
 '895 main st',
 '100 e charlotte st',
 '13210 goodland st',
 '6591 orange dr',
 '1000 broadway drive',
 '333 w lake st

In [279]:
hcpcs10_org['nppes_provider_street1'].value_counts()

5005 sunset blvd            6
617 w main st               4
123 s grace st              4
395 w lake st               4
300 main st                 4
668 hernandez carrion       4
8400 183rd pl               3
1000 battles st             3
10361 spartan dr            3
130 n main st               3
9 main st ste 2k            3
800 main st                 3
100 e main st               3
59 main st                  3
main street                 3
237 oakland dr              3
1 main st                   3
25 main st                  2
284 main st                 2
117 south fayette street    2
110 n main st               2
11606 nicholas st           2
290 armistice blvd          2
63 bo pete manor rd         2
1125 main st                2
1180 n indian canyon dr     2
128 bucksport rd            2
109 n main st               2
2593 w 13th st              2
70 city hall ave            2
                           ..
200 2nd st ne               1
116 michigan ave            1
8214 richa

## Read in the main LEIE file

In [320]:
#leie_m = pd.read_csv('LEIE_ALL_17_19.csv', dtype={'ZIP': object})

leie_m = pd.read_csv('LEIE_exclusions_list_use.csv', dtype={'ZIP': object})

#leie_m = pd.read_csv('LEIE_ALL_17_19', index_col=0)

C:\Users\aparn\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [321]:
leie_m.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN


In [322]:
leie_m.dtypes

LASTNAME       object
FIRSTNAME      object
MIDNAME        object
BUSNAME        object
GENERAL        object
SPECIALTY      object
UPIN           object
NPI             int64
DOB           float64
ADDRESS        object
CITY           object
STATE          object
ZIP            object
EXCLTYPE       object
EXCLDATE        int64
REINDATE        int64
WAIVERDATE      int64
WVRSTATE       object
dtype: object

**Fix the Zip code variable**

In [323]:
## Check the length of the zip code var. Create a variable that stores the length of each zip code and freq it.

leie_m['zip_len']=leie_m['ZIP'].str.len()

In [324]:
# Obtain the frequency counts

leie_m['zip_len'].value_counts()

5    72473
4        3
Name: zip_len, dtype: int64

In [325]:
## Since there are several that are not 5 digit codes, add leading zeros to them. NJ codes have leading zeros.
## This works for the 4 digit codes
## There are 27 that are 3 or 1 digit long. These are problems that we can drop if necessary.

leie_m['zip_new']= leie_m['ZIP'].apply(lambda x: x.zfill(5))

In [326]:
leie_m.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,zip_len,zip_new
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN,5,11787
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN,5,33125
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN,5,10468
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN,5,11234
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN,5,60623


In [327]:
leie_m.shape

(72476, 20)

In [328]:
leie_m.columns.tolist()

['LASTNAME',
 'FIRSTNAME',
 'MIDNAME',
 'BUSNAME',
 'GENERAL',
 'SPECIALTY',
 'UPIN',
 'NPI',
 'DOB',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'EXCLTYPE',
 'EXCLDATE',
 'REINDATE',
 'WAIVERDATE',
 'WVRSTATE',
 'zip_len',
 'zip_new']

**Create two new DFs -- one for Individuals and another for organizations**

**First create a DF for organizations**

In [329]:
## Subset the organizations to examine them more closely without individuals

leie_org = leie_m[leie_m['BUSNAME'].notnull()]

In [330]:
type(leie_org)

pandas.core.frame.DataFrame

In [331]:
leie_org.shape

(3108, 20)

In [332]:
## Get a list of columns with missing values
## Note that this is slightly misleading, since missing NPI is set to zero

leie_org.columns[leie_org.isnull().any()].tolist()

['LASTNAME', 'FIRSTNAME', 'SPECIALTY', 'UPIN', 'DOB', 'ADDRESS', 'WVRSTATE']

In [333]:
## Get a count of the missing values by column

len(leie_org.index) - leie_org.count()

LASTNAME      3108
FIRSTNAME     3108
MIDNAME          0
BUSNAME          0
GENERAL          0
SPECIALTY     2285
UPIN          3095
NPI              0
DOB           3108
ADDRESS          6
CITY             0
STATE            0
ZIP              0
EXCLTYPE         0
EXCLDATE         0
REINDATE         0
WAIVERDATE       0
WVRSTATE      3107
zip_len          0
zip_new          0
dtype: int64

In [334]:
leie_org['BUSNAME'].unique().tolist()

['14 LAWRENCE AVE PHARMACY',
 '143 MEDICAL EQUIPMENT CO',
 '184TH STREET PHARMACY CORP',
 '1951 FLATBUSH AVENUE PHARMACY',
 '1ST COMMUNITY HEALTH CTR, LTD',
 '1ST REHABILITATION OF PORT ST',
 '2001 BATH AVENUE PHARMACY INC',
 '2258 PHARMACY, INC',
 '227 NINTH AVENUE PHARMACY, INC',
 '3839 PHARMACY, INC.',
 '54-26 DRUG CORP',
 '716 TRANSPORTATION, INC',
 '764 FRANKLIN AVE PHARMACY INC',
 '882 E 180TH STREET DRUG CORP',
 'A & A LIVERY, INC',
 'A & A MEDICAL CENTER CORP',
 'A & D PHARMACY, INC',
 'A & H LEE PHARMACY, INC',
 'A & H VITAMINS SUPPLY CORP',
 'A & I HEALTH AIDS',
 'A & J MEDICAL CORP',
 'A & M CARE, INC',
 'A & Y MEDICAL SUPPLY, INC',
 'A AND A ORTHOPEDICS, INC',
 'A BACK & ACCIDENT CHIROPRACTIC',
 'A BASTECKI CHIROPRACTIC',
 'A CARING ALTERNATIVE, INC',
 'A DENTAL CENTER, P C - ALLEN P',
 'A DENTAL CENTER, P C - SOUTHGA',
 'A FAIR DEAL PHARMACY, INC',
 'A FAMILY CARE CENTER',
 'A G  ENTERPRISES, INC',
 "A G O'DELL, INC",
 'A GENTLE DENTAL HOMECARE GROUP',
 'A GIFT FROM CHINA,

In [335]:
leie_org['BUSNAME'].value_counts()

DUNN MEDICAL, INC                 8
LEHIGH VALLEY COMMUNITY MENTAL    4
GOLDSTAR HEALTHCARE, INC          3
MEDICAL ARTS PHARMACY             3
DENTAL HEALTH CARE CLINICS INC    3
SELMA ANCILLARY SERVICES          3
DADE CARE HEALTH CENTER, INC      2
NORTHEAST ARKANSAS AMBULANCE      2
NEW DAY PHARMACY                  2
M & G LIVERY & TRANSPORTATION     2
CENTRAL FLORIDA EAR & HEARING     2
INTERIM HEALTHCARE OF HOLLYWOO    2
NORTHEAST COMMUNITY MENTAL HEA    2
WE CARE HEALTH SUPPLY, INC        2
MEDICINE SHOPPE                   2
BROOKLYN MEDICAL ARTS HIV CARE    2
J'S HOMECARE MEDICAL SERVICES     2
GAINESVILLE PAIN MANAGEMENT       2
MED-CARE DISTRIBUTORS, INC        2
J D'S EAST SYRACUSE PHARMACY      2
PHYSICAL THERAPY & REHAB          2
A-COMMUNITY HOME HEALTH, INC      2
REASON CHIROPRACTIC               2
LA FUENTE OCULAR PROSTHETICS,     2
CLAYWEST HOUSE HEALTHCARE, LLC    2
MISSION HEARING AID CENTER        2
PREDDY'S FUNERAL HOME             2
SAFETY MEDICAL TRANSPORTATIO

**Convert org name to lower case and remove punctuation**

In [336]:
## Convert Org last name to lower case

leie_org['BUSNAME_1']= leie_org['BUSNAME'].apply(lambda x: x.lower())

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [337]:
## Remove punctuations from org name using the string module (trial run):

import string

punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{}~'   # `|` is not present here
transtab = str.maketrans(dict.fromkeys(punct, ''))

leie_org['BUSNAME_1'] = '|'.join(leie_org['BUSNAME_1'].tolist()).translate(transtab).split('|')

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [338]:
leie_org['BUSNAME_1'].unique().tolist()

['14 lawrence ave pharmacy',
 '143 medical equipment co',
 '184th street pharmacy corp',
 '1951 flatbush avenue pharmacy',
 '1st community health ctr ltd',
 '1st rehabilitation of port st',
 '2001 bath avenue pharmacy inc',
 '2258 pharmacy inc',
 '227 ninth avenue pharmacy inc',
 '3839 pharmacy inc',
 '5426 drug corp',
 '716 transportation inc',
 '764 franklin ave pharmacy inc',
 '882 e 180th street drug corp',
 'a  a livery inc',
 'a  a medical center corp',
 'a  d pharmacy inc',
 'a  h lee pharmacy inc',
 'a  h vitamins supply corp',
 'a  i health aids',
 'a  j medical corp',
 'a  m care inc',
 'a  y medical supply inc',
 'a and a orthopedics inc',
 'a back  accident chiropractic',
 'a bastecki chiropractic',
 'a caring alternative inc',
 'a dental center p c  allen p',
 'a dental center p c  southga',
 'a fair deal pharmacy inc',
 'a family care center',
 'a g  enterprises inc',
 'a g odell inc',
 'a gentle dental homecare group',
 'a gift from china inc',
 'a health  stress free',


**Bring in the NLP Stop Words algorithm here to identify only the most important components of the business names**

In [339]:
leie_org['BUSNAME_1'] = leie_org['BUSNAME_1'].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) 

##This splits each string to a list element. This is also called tokenization. It will return a tuple. 

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [340]:
type(leie_org['BUSNAME_1'])

pandas.core.series.Series

In [341]:
## This output shows that the stop words have been removed

leie_org['BUSNAME_1'].unique().tolist()

['14 lawrence ave pharmacy',
 '143 medical equipment co',
 '184th street pharmacy corp',
 '1951 flatbush avenue pharmacy',
 '1st community health ctr ltd',
 '1st rehabilitation port st',
 '2001 bath avenue pharmacy inc',
 '2258 pharmacy inc',
 '227 ninth avenue pharmacy inc',
 '3839 pharmacy inc',
 '5426 drug corp',
 '716 transportation inc',
 '764 franklin ave pharmacy inc',
 '882 e 180th street drug corp',
 'livery inc',
 'medical center corp',
 'pharmacy inc',
 'h lee pharmacy inc',
 'h vitamins supply corp',
 'health aids',
 'j medical corp',
 'care inc',
 'medical supply inc',
 'orthopedics inc',
 'back accident chiropractic',
 'bastecki chiropractic',
 'caring alternative inc',
 'dental center p c allen p',
 'dental center p c southga',
 'fair deal pharmacy inc',
 'family care center',
 'g enterprises inc',
 'g odell inc',
 'gentle dental homecare group',
 'gift china inc',
 'health stress free',
 'k medicar inc',
 'little walnut pharmacy inc',
 'loving touch',
 'perfect smile in

**Split the business name string into multiple words**

In [342]:
## Split the business name string into multiple words, and put each into a column

tempbus = leie_org['BUSNAME_1'].str.split(' ', expand=True).rename(columns = lambda x: "BUSNAME"+str(x+1))

In [343]:
type(tempbus)
tempbus.head()
tempbus.shape

(3108, 7)

In [344]:
#Concatenate the different business name strings with the leie_org dataframe.

##leie_org = pd.concat([leie_org, tempbus], axis = 1) ##Works, but messes up the order of columns.

# Unlike pd.concat, the code below preserves the order of columns in leie_org, and puts the split cols at the end.
leie_org = leie_org.join(tempbus)

In [345]:
leie_org.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,zip_len,zip_new,BUSNAME_1,BUSNAME1,BUSNAME2,BUSNAME3,BUSNAME4,BUSNAME5,BUSNAME6,BUSNAME7
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN,5,11787,14 lawrence ave pharmacy,14,lawrence,ave,pharmacy,None,None,None
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN,5,33125,143 medical equipment co,143,medical,equipment,co,None,None,None
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN,5,10468,184th street pharmacy corp,184th,street,pharmacy,corp,None,None,None
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN,5,11234,1951 flatbush avenue pharmacy,1951,flatbush,avenue,pharmacy,None,None,None
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN,5,60623,1st community health ctr ltd,1st,community,health,ctr,ltd,None,None


**Work with the address string**

In [346]:
leie_org['ADDRESS'].unique().tolist()

['14 LAWRENCE AVENUE',
 '701 NW 36 AVENUE',
 '69 E 184TH ST',
 '1951 FLATBUSH AVE',
 '3138 W CERMAK ROAD',
 'C/O 3659 MAGUIRE BLVD',
 '2001 BATH AVENUE',
 '2258 THIRD AVE',
 '227 9TH AVE',
 '3839 BROADWAY',
 '1120 BROADWAY',
 '540 VINE LANE',
 '764 FRANKLIN AVENUE',
 '882 E 180TH STREET',
 '15 WALLACE STREET',
 '8231 N W 8TH ST, STE 510',
 '467 MYRTLE AVE',
 '268 BROADWAY',
 '592 PALISADE AVENUE',
 '516 SUNNYBROOK DRIVE',
 '2460 W 73RD PLACE',
 'C/O 22 EVERETT AVE',
 '6310 108TH STREET, APT 6J',
 '12250 SW 129TH COURT, BLDG 101',
 '2973 E CHESTNUT EXPRESSWA',
 '123 6TH ST, STE 2',
 '1229 HURON RD E, FLR 6TH',
 '17234 GODDARD ROAD',
 'C/O 7109 WINDING BROOK COURT',
 'C/O  P O BOX 329014, #69709-05',
 '1775 ACCESS RD',
 'ROUTE 2, BOX 386A',
 '12 MAIN STREET',
 '681 BROADWAY AVE',
 '8431 SW 35 TERRACE',
 '2851 N E 183RD ST, #1614',
 '6770 N LINCOLN AVE',
 '10801 NORTH LAMAR',
 '1760 CANDLER RD, STE C',
 '6500 COW PEN RD',
 '3704 AIRLINE DRIVE',
 '4875 N FEDERAL HWY, #9D',
 '13207 S W 46 L

In [347]:
leie_org['ADDRESS'].value_counts()

142 MARION ST                   12
P O BOX 7007, #29361-034         8
4720 GREAT NORTHERN BLVD         8
C/O FEDERAL PRISON CAMP          7
15801 S W 137TH AVE              6
C/O 27 VALLEY ESTATE DRIV        6
C/O BOX 8004, #06269-081         6
1900 SIMLER AVE, FCI#5797        6
C/O 63 E OLD COUNTRY ROAD        6
1841 WYOMING AVE                 6
2999 S VIRGINIA ST               5
C/O P O BOX 600                  5
207 5TH AVENUE, N                5
P O BOX 1000                     5
21300 SW 236 STREET              5
P O BOX 019129, FED DETEN        5
592 PALISADE AVENUE              5
16 FIR DRIVE                     4
C/O P O BOX 1029                 4
6032 S HALSTED                   4
15361 S W 43RD TERRACE           4
HIGHWAY 67 WEST                  4
3775 N W 16TH ST                 4
C/O P O BOX 600, #39781-018      4
7837 10TH AVE, SOUTH             4
917 RED OAK CREEK ROAD           4
C/O 15801 S W 137TH AVE          4
3990 W FLAGLER ST, STE 10        4
110 RABY AVE        

**Split the address string into two parts at the comma to match street 1 and street 2 of CMS files**

In [348]:
## split the address field on the comma and create a temp new data frame with split value columns 
## Splitting is needed to remove things like Suite and Apartment, and move it to a separate column.

new = leie_org['ADDRESS'].str.split(",", n = 1, expand = True) 
  
# make a seprate var for street 1 from the 'new' df 
leie_org["Street_1"]= new[0] 
  
# make a separate var for street 2 from 'new' df
leie_org["Street_2"]= new[1] 

In [349]:
leie_org.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,zip_len,zip_new,BUSNAME_1,BUSNAME1,BUSNAME2,BUSNAME3,BUSNAME4,BUSNAME5,BUSNAME6,BUSNAME7,Street_1,Street_2
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN,5,11787,14 lawrence ave pharmacy,14,lawrence,ave,pharmacy,None,None,None,14 LAWRENCE AVENUE,None
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN,5,33125,143 medical equipment co,143,medical,equipment,co,None,None,None,701 NW 36 AVENUE,None
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN,5,10468,184th street pharmacy corp,184th,street,pharmacy,corp,None,None,None,69 E 184TH ST,None
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN,5,11234,1951 flatbush avenue pharmacy,1951,flatbush,avenue,pharmacy,None,None,None,1951 FLATBUSH AVE,None
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN,5,60623,1st community health ctr ltd,1st,community,health,ctr,ltd,None,None,3138 W CERMAK ROAD,None


In [351]:
leie_org.dtypes

LASTNAME       object
FIRSTNAME      object
MIDNAME        object
BUSNAME        object
GENERAL        object
SPECIALTY      object
UPIN           object
NPI             int64
DOB           float64
ADDRESS        object
CITY           object
STATE          object
ZIP            object
EXCLTYPE       object
EXCLDATE        int64
REINDATE        int64
WAIVERDATE      int64
WVRSTATE       object
zip_len         int64
zip_new        object
BUSNAME_1      object
BUSNAME1       object
BUSNAME2       object
BUSNAME3       object
BUSNAME4       object
BUSNAME5       object
BUSNAME6       object
BUSNAME7       object
Street_1       object
Street_2       object
dtype: object

In [353]:
## Convert Street_1 address to lower case

#leie_org['Street_1']= leie_org['Street_1'].apply(lambda x: x.lower())

leie_org['Street_1'] = leie_org['Street_1'].str.lower()

In [354]:
## Remove punctuations etc from Street_1 name. This version uses Regex.

leie_org['Street_1'] = leie_org['Street_1'].str.replace(r'[^\w\s]+', '')

In [355]:
leie_org.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,zip_len,zip_new,BUSNAME_1,BUSNAME1,BUSNAME2,BUSNAME3,BUSNAME4,BUSNAME5,BUSNAME6,BUSNAME7,Street_1,Street_2
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN,5,11787,14 lawrence ave pharmacy,14,lawrence,ave,pharmacy,None,None,None,14 lawrence avenue,None
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN,5,33125,143 medical equipment co,143,medical,equipment,co,None,None,None,701 nw 36 avenue,None
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN,5,10468,184th street pharmacy corp,184th,street,pharmacy,corp,None,None,None,69 e 184th st,None
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN,5,11234,1951 flatbush avenue pharmacy,1951,flatbush,avenue,pharmacy,None,None,None,1951 flatbush ave,None
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN,5,60623,1st community health ctr ltd,1st,community,health,ctr,ltd,None,None,3138 w cermak road,None


**Read in the individual level file from LEIE**

In [356]:
## Subset the individuals to examine them more closely without organziations

leie_ind = leie_m[leie_m['LASTNAME'].notnull()]

In [357]:
type(leie_ind)

pandas.core.frame.DataFrame

In [358]:
leie_ind.shape

(69365, 20)

In [359]:
# Columns with missings

leie_ind.columns[leie_ind.isnull().any()].tolist()

['FIRSTNAME',
 'BUSNAME',
 'SPECIALTY',
 'UPIN',
 'DOB',
 'ADDRESS',
 'CITY',
 'STATE',
 'WVRSTATE']

In [360]:
# Number missings by column

len(leie_ind.index) - leie_ind.count()

LASTNAME          0
FIRSTNAME         2
MIDNAME           0
BUSNAME       69365
GENERAL           0
SPECIALTY      1915
UPIN          63045
NPI               0
DOB             869
ADDRESS           3
CITY              1
STATE             5
ZIP               0
EXCLTYPE          0
EXCLDATE          0
REINDATE          0
WAIVERDATE        0
WVRSTATE      69353
zip_len           0
zip_new           0
dtype: int64

In [361]:
leie_ind['LASTNAME'].unique().tolist()

['AALDERS',
 'AALTONEN',
 'AAMIR',
 'AARON',
 'AASE',
 'AASEN',
 'ABAD',
 'ABADI',
 'ABADJIVOR',
 'ABADURA',
 'ABAGA',
 'ABAGNALE',
 'ABAIR',
 'ABALOS',
 'ABANZUKWE',
 'ABARCA',
 'ABARQUEZ',
 'ABASHKIN',
 'ABAT',
 'ABATE',
 'ABAYA',
 'ABBARNO',
 'ABBAS',
 'ABBASSI',
 'ABBASSIAN',
 'ABBATE',
 'ABBETT',
 'ABBINANTE',
 'ABBOTT',
 'ABBRUZZESE',
 'ABDALA',
 'ABDAL-ALIM',
 'ABDALLA',
 'ABDALLAH',
 'ABDEL MESSIH',
 'ABDELGAWAD',
 'ABDELHAMID',
 'ABDELRAHMAN',
 'ABDELSHAFI',
 'ABDI',
 'ABDILLE',
 'ABDILNOUR',
 'ABDOO',
 'ABDOYAN',
 'ABDULJABER',
 'ABDULLAH',
 'ABDULLAHI',
 'ABDULLE',
 'ABDULRAHMAN',
 'ABDUR-RAHIM',
 'ABE',
 'ABEENE',
 'ABEITA',
 'ABEL',
 'ABELL',
 'ABELLA',
 'ABELLANOSA',
 'ABENDAN',
 'ABERASTURIA',
 'ABERBACH',
 'ABERCROMBIE',
 'ABERGEL-NAHON',
 'ABERIN',
 'ABERLE',
 'ABERNATHY',
 'ABEYTA',
 'ABIFARIN',
 'ABIGANTUS',
 'ABILA',
 'ABIOYE',
 'ABIR',
 'ABITTAN',
 'ABLANG-TEJADO',
 'ABNER',
 'ABNEY',
 'ABOLAFIA',
 'ABOLAHRAR',
 'ABOLOYE',
 'ABON',
 'ABONITALLA',
 'ABOSHADY',
 'ABO

In [362]:
leie_ind['LASTNAME'].value_counts()

SMITH                  590
JOHNSON                506
WILLIAMS               440
BROWN                  420
JONES                  407
DAVIS                  304
MILLER                 272
THOMAS                 215
JACKSON                215
TAYLOR                 208
MOORE                  204
ANDERSON               198
WILSON                 198
HARRIS                 191
RODRIGUEZ              182
WHITE                  178
MARTIN                 177
LEE                    177
THOMPSON               175
GONZALEZ               162
HERNANDEZ              152
GARCIA                 149
MARTINEZ               148
ROBINSON               144
CLARK                  144
LEWIS                  139
HALL                   130
YOUNG                  125
LOPEZ                  123
ALLEN                  122
                      ... 
FIDLER                   1
HERNANDEZ RIVERA         1
EMERICK                  1
PIECZNSKI                1
TOKARENKO                1
SCHENEWERK               1
B

In [461]:
## split the address field on the comma and create a temp new data frame with split value columns 
## Splitting is needed to remove things like Suite and Apartment, and move it to a separate column.

new = leie_ind['ADDRESS'].str.split(",", n = 1, expand = True) 
  
# make a seprate var for street 1 from the 'new' df 
leie_ind["Street_1"]= new[0] 
  
# make a separate var for street 2 from 'new' df
leie_ind["Street_2"]= new[1] 

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [462]:
## Remove punctuations etc from Street_1 name. This version uses Regex.
leie_ind['Street_1'] = leie_ind['Street_1'].str.lower()

leie_ind['Street_1'] = leie_ind['Street_1'].str.replace(r'[^\w\s]+', '')

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## Check if the individual level file NPIs can be matched with the HCPCS10 file**

In [444]:
x= leie_ind['npi'].value_counts()
x.value_counts()

1        4739
2          94
3           2
64432       1
Name: npi, dtype: int64

**Clean up the name and address fields in both files, since earlier the focus was on the org files**

In [439]:
## Convert the address field to lower case in hcpcs10

hcpcs10_ind['nppes_provider_street1']= hcpcs10_ind['nppes_provider_street1'].str.lower()

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [441]:
## Remove punctuations from address field in hcpcs10

hcpcs10_ind['nppes_provider_street1'] = hcpcs10_ind['nppes_provider_street1'].str.replace(r'[^\w\s]+', '')

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [442]:
hcpcs10_ind['nppes_provider_street1'].unique().tolist()

['900 seton dr',
 '4126 n holland sylvania rd',
 '100 hospital rd',
 '11100 euclid ave',
 '12605 e 16th ave',
 '1565 saxon blvd',
 '1021 park ave',
 '1145 s utica ave',
 '7401 s main',
 '2215 portland ave',
 '8 richland medical park dr',
 '5471 la palma ave',
 '2930 11th ave',
 '1665 s imperial ave ste d',
 '750 e adams st',
 '2401 s 31st st',
 '400 saybrook rd ste 205',
 '1485 river park dr',
 '1245 s utica ave',
 '1397 s linden rd',
 '3200 kearney st',
 '1374 whitehorse hamilton square rd',
 '2640 forest hill blvd',
 '500 congress st ste 1b',
 '400 ne mother joseph pl',
 '801 s bowman rd ste 4',
 '506 this way st',
 '601 elmwood ave',
 '680 guzzi ln',
 '200 patewood dr ste b350',
 '1606 prairie center pkwy',
 'building h100',
 '70 n country rd',
 '904 7th ave',
 '9900 n central expy ste 215',
 '5 roosevelt ave',
 '800 e highway 71',
 '5323 harry hines blvd',
 '333 borthwick ave',
 '3301 s alameda st ste 201',
 '712 south cascade street',
 '550 17th ave',
 '15 w dry creek cir',
 '1700

In [379]:
## Rename the columns of Leie so they match on the hcpcs10 dataframe. This is needed so they can be merged.

leie_ind.rename(columns={'LASTNAME': 'lastname', 'NPI': 'npi'}, inplace=True)

C:\Users\aparn\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [382]:
## Make the hcpcs10 last name column less cumbersome

hcpcs10_ind.rename(columns={'nppes_provider_last_org_name': 'lastname'}, inplace=True)

In [463]:
## Make the hcpcs10 street1 column less cumbersome

hcpcs10_ind.rename(columns={'nppes_provider_street1': 'Street1'}, inplace=True)

In [466]:
## Make the street1 format line up with the format for LEIE file

hcpcs10_ind.rename(columns={'Street1': 'Street_1'}, inplace=True)

**Work on the last name strings in both files**

In [496]:
## Convert the address field to lower case in hcpcs10

hcpcs10_ind['lastname']= hcpcs10_ind['lastname'].str.lower()

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [497]:
## Remove punctuations from address field in hcpcs10

hcpcs10_ind['lastname'] = hcpcs10_ind['lastname'].str.replace(r'[^\w\s]+', '')

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [499]:
## Convert the address field to lower case in hcpcs10

leie_ind['lastname']= leie_ind['lastname'].str.lower()

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [500]:
## Remove punctuations from address field in hcpcs10

leie_ind['lastname'] = leie_ind['lastname'].str.replace(r'[^\w\s]+', '')

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [501]:
hcpcs10_ind.head()

,npi,lastname,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,Street_1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,zip_len,prov_zip,nzip_len,zip_new,nzip2_len
6,1003000126,enkeshafi,ARDALAN,NaN,M.D.,M,I,900 seton dr,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,NaN,21502,5,21502,5
7,1003000126,enkeshafi,ARDALAN,NaN,M.D.,M,I,900 seton dr,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,NaN,21502,5,21502,5
8,1003000126,enkeshafi,ARDALAN,NaN,M.D.,M,I,900 seton dr,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,NaN,21502,5,21502,5
10,1003000126,enkeshafi,ARDALAN,NaN,M.D.,M,I,900 seton dr,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,NaN,21502,5,21502,5
39,1003000142,khalil,RASHID,NaN,M.D.,M,I,4126 n holland sylvania rd,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,NaN,43623,5,43623,5


In [502]:
leie_ind.head()

,lastname,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,npi,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,zip_len,zip_new,Street_1,Street_2
3108,aalders,NANCY,BLOCK,NaN,HOSPITAL,NURSE/NURSES AIDE,NaN,0,19640305.0,9527 SOUTH KENTON,OAK LAWN,IL,60453,1128b4,20030820,0,0,NaN,5,60453,9527 south kenton,None
3109,aaltonen,NICKOLAS,A,NaN,IND- LIC HC SERV PRO,PHARMACY TECHNICIAN,NaN,0,19880123.0,32 W PACIFIC AVE,SPOKANE,WA,99201,1128a4,20120419,0,0,NaN,5,99201,32 w pacific ave,None
3110,aamir,MUHAMMAD,SOHAIL,NaN,BUS OWNER/EXEC,HOME HEALTH AGENCY,NaN,0,19700910.0,3956 COLUMBIA COURT,BLOOMFIELD HILLS,MI,48302,1128a1,20170220,0,0,NaN,5,48302,3956 columbia court,None
3111,aaron,ALINA,A,NaN,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,NaN,0,19870209.0,3325 N ROSE CIRCLE DRIVE,PHOENIX,AZ,85018,1128b4,20171019,0,0,NaN,5,85018,3325 n rose circle drive,None
3112,aaron,BARBARA,KAY,NaN,NURSING PROFESSION,NURSE/NURSES AIDE,NaN,0,19650914.0,5911 OLD PINEYWOODS RD,JASPER,AL,35504,1128b4,20061120,0,0,NaN,5,35504,5911 old pineywoods rd,None


**Check if there are matches between the two files even before subsetting data**

In [503]:
hcpcs10_ind['lastname'].isin(leie_ind['lastname']).value_counts()

True     834479
False    677460
Name: lastname, dtype: int64

In [504]:
hcpcs10_ind['npi'].isin(leie_ind['npi']).value_counts()

False    1511662
True         277
Name: npi, dtype: int64

In [505]:
hcpcs10_ind['zip_new'].isin(leie_ind['zip_new']).value_counts()

True     1279532
False     232407
Name: zip_new, dtype: int64

In [506]:
#list(leie_ind.columns)
leie_ind.columns

Index(['lastname', 'FIRSTNAME', 'MIDNAME', 'BUSNAME', 'GENERAL', 'SPECIALTY',
       'UPIN', 'npi', 'DOB', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'EXCLTYPE',
       'EXCLDATE', 'REINDATE', 'WAIVERDATE', 'WVRSTATE', 'zip_len', 'zip_new',
       'Street_1', 'Street_2'],
      dtype='object')

In [507]:
hcpcs10_ind.columns

Index(['npi', 'lastname', 'nppes_provider_first_name', 'nppes_provider_mi',
       'nppes_credentials', 'nppes_provider_gender', 'nppes_entity_code',
       'Street_1', 'nppes_provider_street2', 'nppes_provider_city',
       'nppes_provider_zip', 'nppes_provider_state', 'nppes_provider_country',
       'provider_type', 'medicare_participation_indicator', 'place_of_service',
       'hcpcs_code', 'hcpcs_description', 'hcpcs_drug_indicator',
       'line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt',
       'average_Medicare_allowed_amt', 'average_submitted_chrg_amt',
       'average_Medicare_payment_amt', 'average_Medicare_standard_amt',
       'Total_submitted_chrg_amt', 'Total_Medicare_payment_amt', 'zip_len',
       'prov_zip', 'nzip_len', 'zip_new', 'nzip2_len'],
      dtype='object')

**Double check the missings in both files**

In [508]:
leie_ind.columns[leie_ind.isnull().any()].tolist()

['FIRSTNAME',
 'BUSNAME',
 'SPECIALTY',
 'UPIN',
 'DOB',
 'ADDRESS',
 'CITY',
 'STATE',
 'WVRSTATE',
 'Street_1',
 'Street_2']

In [509]:
# Number missings by column

len(leie_ind.index) - leie_ind.count()

lastname          0
FIRSTNAME         2
MIDNAME           0
BUSNAME       69365
GENERAL           0
SPECIALTY      1915
UPIN          63045
npi               0
DOB             869
ADDRESS           3
CITY              1
STATE             5
ZIP               0
EXCLTYPE          0
EXCLDATE          0
REINDATE          0
WAIVERDATE        0
WVRSTATE      69353
zip_len           0
zip_new           0
Street_1          3
Street_2      50359
dtype: int64

In [510]:
hcpcs10_ind.columns[hcpcs10_ind.isnull().any()].tolist()

['lastname',
 'nppes_provider_first_name',
 'nppes_provider_mi',
 'nppes_credentials',
 'Street_1',
 'nppes_provider_street2',
 'zip_len']

In [511]:
# Number missings by column

len(hcpcs10_ind.index) - hcpcs10_ind.count()

npi                                      0
lastname                                26
nppes_provider_first_name               24
nppes_provider_mi                   436779
nppes_credentials                    56621
nppes_provider_gender                    0
nppes_entity_code                        0
Street_1                                 2
nppes_provider_street2              880777
nppes_provider_city                      0
nppes_provider_zip                       0
nppes_provider_state                     0
nppes_provider_country                   0
provider_type                            0
medicare_participation_indicator         0
place_of_service                         0
hcpcs_code                               0
hcpcs_description                        0
hcpcs_drug_indicator                     0
line_srvc_cnt                            0
bene_unique_cnt                          0
bene_day_srvc_cnt                        0
average_Medicare_allowed_amt             0
average_sub

**Begin the process of subsetting files to identify the unique matches between the two files**

In [512]:
## remove into new dataframe, those columns in hcpcs10_ind that match npis in leie

dfnpi = hcpcs10_ind[hcpcs10_ind.npi.isin(leie_ind.npi.values)]

In [513]:
type(dfnpi)
dfnpi.shape
dfnpi.npi.nunique()

137

In [514]:
## remove into new dataframe, those columns in hcpcs10_ind that do not match npis in leie

newdf = hcpcs10_ind[~hcpcs10_ind.npi.isin(leie_ind.npi.values)]

In [515]:
type(newdf)
newdf.shape
newdf.npi.nunique()

652340

In [516]:
newdf['zip_new'].isin(leie_ind['zip_new']).value_counts()

True     1279283
False     232379
Name: zip_new, dtype: int64

In [517]:
## remove into new dataframe, those columns in newdf that match npis in leie

dfzip = newdf[newdf.zip_new.isin(leie_ind.zip_new.values)]

In [518]:
type(dfzip)
dfzip.shape
dfzip.zip_new.nunique()

12269

In [519]:
dfzip['lastname'].isin(leie_ind['lastname']).value_counts()

True     708790
False    570493
Name: lastname, dtype: int64

In [520]:
## remove into new dataframe, those columns in dfzip that match lastnames in leie

dfname = dfzip[newdf.lastname.isin(leie_ind.lastname.values)]

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [521]:
type(dfname)
dfname.shape
dfname.lastname.nunique()

16847

In [482]:
dfname.head()

,npi,lastname,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,Street_1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,zip_len,prov_zip,nzip_len,zip_new,nzip2_len
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 n holland sylvania rd,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.93,109.0,50.868062,53.268372,14061.0,6561.98,NaN,43623,5,43623,5
40,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 n holland sylvania rd,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,F,99214,Established patient office or other outpatient...,N,24.0,24,24,77.84,115.0,58.658333,63.761667,2760.0,1407.80,NaN,43623,5,43623,5
41,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 n holland sylvania rd,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99214,Established patient office or other outpatient...,N,330.0,140,330,104.53,159.0,73.218515,78.293848,52470.0,24162.11,NaN,43623,5,43623,5
117,1003000597,KIM,DAE,Y,"M.D., PH.D",M,I,1145 s utica ave,SUITE 202,TULSA,741044000,OK,US,Urology,Y,O,99213,Established patient office or other outpatient...,N,320.0,237,320,69.36,150.0,50.444219,54.966094,48000.0,16142.15,NaN,74104,5,74104,5
118,1003000597,KIM,DAE,Y,"M.D., PH.D",M,I,1145 s utica ave,SUITE 202,TULSA,741044000,OK,US,Urology,Y,O,99214,Established patient office or other outpatient...,N,255.0,195,255,102.33,224.0,70.698000,76.838039,57120.0,18027.99,NaN,74104,5,74104,5


In [522]:
## remove into new dataframe, those columns in dfname that match street 1 addresses in leie

dfname['Street_1'].isin(leie_ind['Street_1']).value_counts()

False    705187
True       3603
Name: Street_1, dtype: int64

In [523]:
dfstreet = dfname[dfname.Street_1.isin(leie_ind.Street_1.values)]

In [524]:
type(dfstreet)
dfstreet.shape
dfstreet.Street_1.nunique()

443

In [525]:
dfstreet.head(20)

,npi,lastname,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,Street_1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,zip_len,prov_zip,nzip_len,zip_new,nzip2_len
7901,1003044611,danek,SAMANTHA,R,PA-C,F,I,205 n east ave,NaN,JACKSON,492011753,MI,US,Physician Assistant,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,1020.0,132,1020,60.830000,165.735294,47.564696,48.809706,169050.000002,48515.990000,NaN,49201,5,49201,5
7902,1003044611,danek,SAMANTHA,R,PA-C,F,I,205 n east ave,NaN,JACKSON,492011753,MI,US,Physician Assistant,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,114.0,40,114,87.960000,246.000000,68.355088,69.850965,28044.000000,7792.480000,NaN,49201,5,49201,5
19615,1003125485,hanna,EHAB,S,M.D.,M,I,301 e main st,NaN,BAY SHORE,117068408,NY,US,Hospitalist,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,321.0,308,321,235.780000,879.000000,178.010997,154.381215,282159.000000,57141.529998,NaN,11706,5,11706,5
27779,1003230798,dinh,DUY,NaN,PA-C,M,I,8631 w 3rd st,715E,LOS ANGELES,900485901,CA,US,Physician Assistant,Y,F,99284,"Emergency department visit, problem of high se...",N,160.0,157,160,104.540000,446.687500,76.084438,73.652438,71470.000000,12173.510000,NaN,90048,5,90048,5
27780,1003230798,dinh,DUY,NaN,PA-C,M,I,8631 w 3rd st,715E,LOS ANGELES,900485901,CA,US,Physician Assistant,Y,F,99285,"Emergency department visit, problem with signi...",N,162.0,160,162,154.050000,671.962963,113.061790,110.153704,108858.000000,18316.009999,NaN,90048,5,90048,5
50711,1003836248,wagner,WILLIS,H,M.D.,M,I,8631 w 3rd st,SUITE 615E,LOS ANGELES,900485901,CA,US,Vascular Surgery,Y,O,99213,Established patient office or other outpatient...,N,78.0,71,78,79.620000,100.000000,58.698462,55.609359,7800.000000,4578.480000,NaN,90048,5,90048,5
50712,1003836248,wagner,WILLIS,H,M.D.,M,I,8631 w 3rd st,SUITE 615E,LOS ANGELES,900485901,CA,US,Vascular Surgery,Y,O,99214,Established patient office or other outpatient...,N,26.0,24,26,116.850000,125.000000,74.720769,72.606154,3250.000000,1942.740000,NaN,90048,5,90048,5
91058,1003951013,hussain,YUSRA,N,M.D.,F,I,770 welch rd,SUITE # 250,PALO ALTO,943041511,CA,US,Internal Medicine,Y,O,99213,Established patient office or other outpatient...,N,186.0,57,186,88.270000,95.779570,62.279140,52.126882,17815.000000,11583.920000,NaN,94304,5,94304,5
91059,1003951013,hussain,YUSRA,N,M.D.,F,I,770 welch rd,SUITE # 250,PALO ALTO,943041511,CA,US,Internal Medicine,Y,O,99214,Established patient office or other outpatient...,N,125.0,48,125,129.240000,135.080000,91.228880,76.448080,16885.000000,11403.610000,NaN,94304,5,94304,5
91061,1003951013,hussain,YUSRA,N,M.D.,F,I,770 welch rd,SUITE # 250,PALO ALTO,943041511,CA,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,17.0,11,17,82.630000,100.058824,64.780000,57.390000,1701.000000,1101.260000,NaN,94304,5,94304,5
